In [1]:
DATA_FOLDER = 'data/so/'

In [2]:
from IPython.display import display, HTML
import numpy as np

Read data

In [3]:
file_index = 1
events = {}
times = {}

for data_type in ['train', 'test']:
    with open(DATA_FOLDER + 'event-' + str(file_index) + '-' + data_type + '.txt') as f_events:
        events[data_type] = [[int(y) for y in x.split()] for x in f_events]
        
    with open(DATA_FOLDER + 'time-' + str(file_index) + '-' + data_type + '.txt') as f_times:
        times[data_type] = [[float(y) for y in x.split()] for x in f_times]

badge_map = {}
with open(DATA_FOLDER + 'badges.csv') as f_badge_labels:
    next(f_badge_labels)
    for row in f_badge_labels:
        id, name = row.split(',')
        badge_map[int(id)] = name.strip()

Compute deltas for times.

It seems that the general approach usually is to just use timestamps, but going to also compute deltas because maybe will want to use that to make things easier.

In [4]:
deltas = {}
for data_type in ['train', 'test']:
    deltas[data_type] = []
    for seq in times[data_type]:
        seq_deltas = []
        for i in range(1, len(seq)):
            seq_deltas.append(seq[i] - seq[i-1])
        deltas[data_type].append(seq_deltas)

# Models for Time

Errors for these models is RMSE:

https://en.wikipedia.org/wiki/Root-mean-square_deviation

## Get Time Stats

In [5]:
all_times = []

for data_type in ['train', 'test']:
    for seq in times[data_type]:
        all_times += seq

print('Max Time', '\t', "{:,}".format(max(all_times)))
print('Min Time', '\t', "{:,}".format(min(all_times)))
print('Mean', '\t\t', "{:,}".format(np.mean(all_times)))
print('Std Dev', '\t', "{:,}".format(np.std(all_times)))
print('')

sorted_times = sorted(all_times, reverse=True)
print('---')
print('Top 10 largest times')
print(sorted_times[:10])

sorted_times = sorted(all_times, reverse=False)
print('')
print('Top 10 smallest times')
print(sorted_times[:10])

Max Time 	 1,388,534,141.777
Min Time 	 1,325,376,181.507
Mean 		 1,359,084,129.1882882
Std Dev 	 17,138,171.0850439

---
Top 10 largest times
[1388534141.777, 1388534141.777, 1388534141.777, 1388534141.777, 1388534141.777, 1388533810.627, 1388533518.993, 1388533207.767, 1388532906.527, 1388532906.527]

Top 10 smallest times
[1325376181.507, 1325376183.3, 1325376480.743, 1325379503.237, 1325380415.083, 1325383721.89, 1325384932.437, 1325385234.173, 1325386138.81, 1325387337.333]


## Get Delta Stats

In [6]:
all_deltas = []

for data_type in ['train', 'test']:
    for seq in deltas[data_type]:
        all_deltas += seq

print('Max Delta', '\t', "{:,}".format(max(all_deltas)))
print('Min Delta', '\t', "{:,}".format(min(all_deltas)))
print('Mean', '\t\t', "{:,}".format(np.mean(all_deltas)))
print('Std Dev', '\t', "{:,}".format(np.std(all_deltas)))
print('')

sorted_deltas = sorted(all_deltas, reverse=True)
print('---')
print('Top 10 largest deltas')
print(sorted_deltas[:10])

sorted_deltas = sorted(all_deltas, reverse=False)
print('')
print('Top 10 smallest deltas')
print(sorted_deltas[:10])

Max Delta 	 20,337,973.434000015
Min Delta 	 0.013000011444091797
Mean 		 826,231.5698269516
Std Dev 	 1,043,219.0273143519

---
Top 10 largest deltas
[20337973.434000015, 20061996.75300002, 19054174.663000107, 18746822.46299982, 18725172.24000001, 17476425.897000074, 17350034.782999992, 17132427.25999999, 16679536.736999989, 16436643.904000044]

Top 10 smallest deltas
[0.013000011444091797, 0.013000011444091797, 0.01399993896484375, 0.01399993896484375, 0.01399993896484375, 0.014000177383422852, 0.016000032424926758, 0.016000032424926758, 0.016000032424926758, 0.016000032424926758]


# Helper Functions

In [7]:
def make_flat(arr):
    flattened = []
    for seq in arr:
        flattened += seq
        
    return flattened

In [8]:
from sklearn.metrics import mean_squared_error
import math

def compute_error(y_true, y_pred):
    y_true = make_flat(y_true)
    y_pred = make_flat(y_pred)
    
    return math.sqrt(mean_squared_error(y_true, y_pred))

def show_results(train_pred, test_pred):
    print('----')
    print('Train Error: ', compute_error(deltas['train'], train_pred))
    print('Test Error: ', compute_error(deltas['test'], test_pred))

# Baseline Predictor

Using methodology from https://github.com/dunan/NeuralPointProcess/blob/master/code/baselines/majority_predictor/time_mean_baseline.py

A scale is used there so adding a variable for that.

In [9]:
scale = 0.001

In [10]:
total = 0
num_train = 0
for seq in times['train']:
    for i in range(1, len(seq)):
        total += scale * (seq[i] - seq[i-1])
        num_train += 1

mean_predictor = total / num_train
print('Mean predictor', mean_predictor)

train_pred = [[mean_predictor for d in seq[1:]] for seq in times['train']]
test_pred = [[mean_predictor for d in seq[1:]] for seq in times['test']]
show_results(train_pred, test_pred)

Mean predictor 828.9236502234086
----
Train Error:  1332529.9103083226
Test Error:  1321285.6966187225


Trying an alternate way by using the deltas to see if different results are obtained.

In [11]:
train_deltas = make_flat(deltas['train'])
train_deltas = np.asarray(train_deltas) * scale

mean_delta = np.mean(train_deltas)
print('Mean predictor using deltas', mean_delta)

train_pred = [[mean_delta for d in seq] for seq in deltas['train']]
test_pred = [[mean_delta for d in seq] for seq in deltas['test']]
show_results(train_pred, test_pred)

Mean predictor using deltas 828.923650223
----
Train Error:  1332529.9103083226
Test Error:  1321285.6966187225


# Structure Data

In [12]:
x = {}
y = {}

for data_type in ['train', 'test']:
    x[data_type] = [seq[:-1] for seq in deltas[data_type]]
    y[data_type] = [seq[1:] for seq in deltas[data_type]]

    x[data_type] = np.asarray(x[data_type])
    y[data_type] = np.asarray(y[data_type])

In [13]:
# Find the max user length
user_lengths = [len(seq) for seq in deltas['train']] + [len(seq) for seq in deltas['test']]
user_length_counts = {}
for length in user_lengths:
    if length not in user_length_counts:
        user_length_counts[length] = 0
        
    user_length_counts[length] += 1
user_lengths = list(set(user_lengths))
user_lengths = sorted(user_lengths, reverse=True)
max_user_length = max(user_lengths)

print('Max Seq Length', max_user_length)
# print(user_lengths)

Max Seq Length 735


In [14]:
# Pad sequences
from keras.preprocessing import sequence

for data_type in ['train', 'test']:
    x[data_type] = sequence.pad_sequences(x[data_type], maxlen=max_user_length)
    y[data_type] = sequence.pad_sequences(y[data_type], maxlen=max_user_length)

Using TensorFlow backend.


# RNN Approaches (Without Events)

In [ ]:
from keras.models import Sequential, Model
from keras import optimizers
from keras.layers import Input, Dense, TimeDistributed, Masking, SimpleRNN, Concatenate

## RNN Cells

In [ ]:
input_events = Input(shape=(max_user_length, 1))
input_deltas = Input(shape=(max_user_length, 1))

inputs = Concatenate(axis=2)([input_events, input_deltas])
inputs = Masking(mask_value=0.)(inputs)

rnn = SimpleRNN(50, activation='tanh', return_sequences=True)(inputs)

output_events = TimeDistributed(Dense(num_events, activation='softmax'), name='events')(rnn)

output_deltas = TimeDistributed(Dense(15, activation='relu'))(rnn)
output_deltas = TimeDistributed(Dense(1, activation='sigmoid'), name='time')(output_deltas)

model = Model(inputs=[input_events, input_deltas], outputs=[output_events, output_deltas])

model.compile(loss=['categorical_crossentropy', 'mse'], optimizer='adam', metrics={'events': 'accuracy'})

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=3)

preds_train = model.predict(x['train'])
preds_test = model.predict(x['test'])

preds_train[1] *= max_delta
preds_test[1] *= max_delta

show_results_nn(preds_train, preds_test)

## LSTM Cells

# WTTE-RNN

## Structure Data

In [47]:
x = {}
y = {}

for data_type in ['train', 'test']:
    # x[data_type] = [seq[:-1] for seq in deltas[data_type]]
    x[data_type] = [[1 for event in seq[:-1]] for seq in deltas[data_type]]
    y[data_type] = [seq[1:] for seq in deltas[data_type]]

    x[data_type] = np.asarray(x[data_type])
    y[data_type] = np.asarray(y[data_type])

In [48]:
# Find the max user length
user_lengths = [len(seq) for seq in deltas['train']] + [len(seq) for seq in deltas['test']]
user_length_counts = {}
for length in user_lengths:
    if length not in user_length_counts:
        user_length_counts[length] = 0
        
    user_length_counts[length] += 1
user_lengths = list(set(user_lengths))
user_lengths = sorted(user_lengths, reverse=True)
max_user_length = max(user_lengths)

print('Max Seq Length', max_user_length)
# print(user_lengths)

Max Seq Length 735


In [49]:
# Pad sequences
from keras.preprocessing import sequence

for data_type in ['train', 'test']:
    x[data_type] = sequence.pad_sequences(x[data_type], maxlen=max_user_length)
    y[data_type] = sequence.pad_sequences(y[data_type], maxlen=max_user_length)

In [55]:
for data_type in ['train', 'test']:
    # x[data_type] = np.expand_dims(x[data_type], axis=2)
    # y[data_type] = np.expand_dims(y[data_type], axis=2)
    
    y_shape = y[data_type].shape
    y_add = np.zeros((y_shape[0], y_shape[1], 1))
    y[data_type] = np.concatenate((y[data_type], y_add), axis=2)

---
(5307, 735, 1)
(5307, 735, 1)
(5307, 735, 2)
---
(1326, 735, 1)
(1326, 735, 1)
(1326, 735, 2)


In [56]:
y['train'].shape

(5307, 735, 2)

Add in a censored value

In [19]:
from keras import backend as K

def _keras_unstack_hack(ab):
    """Implements tf.unstack(y_true_keras, num=2, axis=-1).
       Keras-hack adopted to be compatible with theano backend.
    """
    ndim = len(K.int_shape(ab))
    if ndim == 0:
        print('can not unstack with ndim=0')
    else:
        a = ab[..., 0]
        b = ab[..., 1]
    return a, b

def weibull_loss_discrete(y_true, y_pred, name=None):
    """calculates a keras loss op designed for the sequential api.
    
        Discrete log-likelihood for Weibull hazard function on censored survival data.
        For math, see 
        https://ragulpr.github.io/assets/draft_master_thesis_martinsson_egil_wtte_rnn_2016.pdf (Page 35)
        
        Args:
            y_true: tensor with last dimension having length 2
                with y_true[:,...,0] = time to event, 
                     y_true[:,...,1] = indicator of not censored
                
            y_pred: tensor with last dimension having length 2 
                with y_pred[:,...,0] = alpha, 
                     y_pred[:,...,1] = beta

        Returns:
            A positive `Tensor` of same shape as input
            
    """    
    y,u = _keras_unstack_hack(y_true)
    a,b = _keras_unstack_hack(y_pred)

    hazard0 = K.pow((y + 1e-35) / a, b)
    hazard1 = K.pow((y + 1.0) / a, b)
    
    loglikelihoods = u * K.log(K.exp(hazard1 - hazard0) - 1.0) - hazard1
    loss = -1 * K.mean(loglikelihoods)
    return loss


def output_lambda(x, init_alpha=1.0, max_beta_value=5.0, max_alpha_value=None):
    """Elementwise (Lambda) computation of alpha and regularized beta.

        Alpha: 
        (activation) 
        Exponential units seems to give faster training than 
        the original papers softplus units. Makes sense due to logarithmic
        effect of change in alpha. 
        (initialization) 
        To get faster training and fewer exploding gradients,
        initialize alpha to be around its scale when beta is around 1.0,
        approx the expected value/mean of training tte. 
        Because we're lazy we want the correct scale of output built
        into the model so initialize implicitly; 
        multiply assumed exp(0)=1 by scale factor `init_alpha`.

        Beta: 
        (activation) 
        We want slow changes when beta-> 0 so Softplus made sense in the original 
        paper but we get similar effect with sigmoid. It also has nice features.
        (regularization) Use max_beta_value to implicitly regularize the model
        (initialization) Fixed to begin moving slowly around 1.0

        Assumes tensorflow backend.

        Args:
            x: tensor with last dimension having length 2
                with x[...,0] = alpha, x[...,1] = beta

        Usage:
            model.add(Dense(2))
            model.add(Lambda(output_lambda, arguments={"init_alpha":100., "max_beta_value":2.0}))
        Returns:
            A positive `Tensor` of same shape as input
    """
    a, b = _keras_unstack_hack(x)

    # Implicitly initialize alpha:
    if max_alpha_value is None:
        a = init_alpha * K.exp(a)
    else:
        a = init_alpha * K.clip(x=a, min_value=K.epsilon(),
                                max_value=max_alpha_value)

    m = max_beta_value
    if m > 1.05:  # some value >>1.0
        # shift to start around 1.0
        # assuming input is around 0.0
        _shift = np.log(m - 1.0)

        b = K.sigmoid(b - _shift)
    else:
        b = K.sigmoid(b)

    # Clipped sigmoid : has zero gradient at 0,1
    # Reduces the small tendency of instability after long training
    # by zeroing gradient.
    b = m * K.clip(x=b, min_value=K.epsilon(), max_value=1. - K.epsilon())

    x = K.stack([a, b], axis=-1)

    return x

In [20]:
import numpy as np

In [21]:
def get_data(n_timesteps, every_nth,n_repeats,noise_level,n_features,use_censored = True):
    def get_equal_spaced(n, every_nth):
        # create some simple data of evenly spaced events recurring every_nth step
        # Each is on (time,batch)-format
        events = np.array([np.array(range(n)) for _ in range(every_nth)])
        events = events + np.array(range(every_nth)).reshape(every_nth, 1) + 1

        tte_actual = every_nth - 1 - events % every_nth

        was_event = (events % every_nth == 0) * 1.0
        was_event[:, 0] = 0.0

        events = tte_actual == 0

        is_censored = (events[:, ::-1].cumsum(1)[:, ::-1] == 0) * 1
        tte_censored = is_censored[:, ::-1].cumsum(1)[:, ::-1] * is_censored
        tte_censored = tte_censored + (1 - is_censored) * tte_actual

        events = np.copy(events.T * 1.0)
        tte_actual = np.copy(tte_actual.T * 1.0)
        tte_censored = np.copy(tte_censored.T * 1.0)
        was_event = np.copy(was_event.T * 1.0)
        not_censored = 1 - np.copy(is_censored.T * 1.0)

        return tte_censored, not_censored, was_event, events, tte_actual
    
    tte_censored,not_censored,was_event,events,tte_actual = get_equal_spaced(n=n_timesteps,every_nth=every_nth)

    # From https://keras.io/layers/recurrent/
    # input shape rnn recurrent if return_sequences: (nb_samples, timesteps, input_dim)

    u_train      = not_censored.T.reshape(n_sequences,n_timesteps,1)
    x_train      = was_event.T.reshape(n_sequences,n_timesteps,1)
    tte_censored = tte_censored.T.reshape(n_sequences,n_timesteps,1)
    y_train      = np.append(tte_censored,u_train,axis=2) # (n_sequences,n_timesteps,2)

    u_test       = np.ones(shape=(n_sequences,n_timesteps,1))
    x_test       = np.copy(x_train)
    tte_actual   = tte_actual.T.reshape(n_sequences,n_timesteps,1)
    y_test       = np.append(tte_actual,u_test,axis=2) # (n_sequences,n_timesteps,2)

    if not use_censored:
        x_train = np.copy(x_test)
        y_train = np.copy(y_test)
    # Since the above is deterministic perfect fit is feasible. 
    # More noise->more fun so add noise to the training data:
    
    x_train = np.tile(x_train.T,n_repeats).T
    y_train = np.tile(y_train.T,n_repeats).T

    # Try with more than one feature TODO
    x_train_new = np.zeros([x_train.shape[0],x_train.shape[1],n_features])
    x_test_new = np.zeros([x_test.shape[0],x_test.shape[1],n_features])
    for f in range(n_features):
        x_train_new[:,:,f] = x_train[:,:,0]
        x_test_new[:,:,f]  = x_test[:,:,0]
        
    x_train = x_train_new
    x_test  = x_test_new
    
    # xtrain is signal XOR noise with probability noise_level
    noise = np.random.binomial(1,noise_level,size=x_train.shape)
    x_train = x_train+noise-x_train*noise
    return y_train,x_train, y_test,x_test,events


n_timesteps    = 200
n_sequences = every_nth = 80
n_features = 1
n_repeats = 1000
noise_level = 0.005
use_censored = True

y_train,x_train, y_test,x_test,events = get_data(n_timesteps, every_nth,n_repeats,noise_level,n_features,use_censored)

In [41]:
from keras.models import Sequential
from keras.layers import Dense

from keras.layers import LSTM,GRU
from keras.layers import Lambda
from keras.layers.wrappers import TimeDistributed

from keras.optimizers import RMSprop,adam
from keras.callbacks import History, TensorBoard

Example

In [42]:
print(y['train'].shape)
print(y_train.shape)

(5307, 735)
(80000, 200, 2)


In [ ]:
tte_mean_train = np.nanmean(y_train[:,:,0])
init_alpha = -1.0/np.log(1.0-1.0/(tte_mean_train+1.0) )
init_alpha = init_alpha/np.nanmean(y_train[:,:,1]) # use if lots of censoring
print('init_alpha: ', init_alpha)

# Start building the model
model = Sequential()
#model.add(TimeDistributed(Dense(2), input_shape=(None, n_features)))
model.add(GRU(1, input_shape=(n_timesteps, n_features),activation='tanh',return_sequences=True))

model.add(TimeDistributed(Dense(2)))
model.add(TimeDistributed(Lambda(output_lambda, arguments={"init_alpha":init_alpha, 
                                               "max_beta_value":4.0})))

model.compile(loss=weibull_loss_discrete, optimizer=adam(lr=.01))

model.summary()

Mine

In [59]:
tte_mean_train = np.nanmean(y['train'][:,:,0])
init_alpha = -1.0/np.log(1.0-1.0/(tte_mean_train+1.0) )
init_alpha = init_alpha/np.nanmean(y['train'][:,:,1]) # use if lots of censoring
print('init_alpha: ', init_alpha)

# Start building the model
model = Sequential()
#model.add(TimeDistributed(Dense(2), input_shape=(None, n_features)))
model.add(GRU(1, input_shape=(x['train'].shape[1], n_features),activation='tanh',return_sequences=True))

model.add(TimeDistributed(Dense(2)))
model.add(TimeDistributed(Lambda(output_lambda, arguments={"init_alpha":init_alpha, 
                                               "max_beta_value":4.0})))

model.compile(loss=weibull_loss_discrete, optimizer=adam(lr=.01))

model.summary()

//anaconda/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


init_alpha:  inf
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 735, 1)            9         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 735, 2)            4         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 735, 2)            0         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.fit(x['train'], y['train'],
          epochs=80, 
          batch_size=x['train'].shape[0]//10, 
          verbose=2, 
#           validation_data=(x_test, y_test)
         )

Epoch 1/80
 - 12s - loss: nan
Epoch 2/80
 - 12s - loss: nan
Epoch 3/80


KeyboardInterrupt: 

In [ ]:
# preds = model.predict(x_test)

# model.fit(x_train, y_train,
#           epochs=80, 
#           batch_size=x_train.shape[0]//10, 
#           verbose=2, 
#           validation_data=(x_test, y_test)
#          )